# Rangkaian Generatif

Recurrent Neural Networks (RNNs) dan varian sel berpagar seperti Long Short Term Memory Cells (LSTMs) dan Gated Recurrent Units (GRUs) menyediakan mekanisme untuk pemodelan bahasa, iaitu mereka boleh mempelajari susunan perkataan dan memberikan ramalan untuk perkataan seterusnya dalam satu urutan. Ini membolehkan kita menggunakan RNN untuk **tugas generatif**, seperti penjanaan teks biasa, terjemahan mesin, dan juga kapsyen imej.

Dalam seni bina RNN yang kita bincangkan dalam unit sebelumnya, setiap unit RNN menghasilkan keadaan tersembunyi seterusnya sebagai output. Walau bagaimanapun, kita juga boleh menambah satu lagi output kepada setiap unit berulang, yang membolehkan kita menghasilkan **urutan** (yang sama panjang dengan urutan asal). Selain itu, kita boleh menggunakan unit RNN yang tidak menerima input pada setiap langkah, dan hanya mengambil beberapa vektor keadaan awal, kemudian menghasilkan urutan output.

Dalam notebook ini, kita akan memberi tumpuan kepada model generatif mudah yang membantu kita menjana teks. Untuk kesederhanaan, mari bina **rangkaian peringkat aksara**, yang menjana teks huruf demi huruf. Semasa latihan, kita perlu mengambil beberapa korpus teks, dan membahagikannya kepada urutan huruf.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

## Membina Perbendaharaan Kata Karakter

Untuk membina rangkaian generatif pada tahap karakter, kita perlu memecahkan teks kepada karakter individu dan bukannya perkataan. Lapisan `TextVectorization` yang telah kita gunakan sebelum ini tidak dapat melakukannya, jadi kita mempunyai dua pilihan:

* Memuatkan teks secara manual dan melakukan tokenisasi 'secara manual', seperti dalam [contoh rasmi Keras ini](https://keras.io/examples/generative/lstm_character_level_text_generation/)
* Menggunakan kelas `Tokenizer` untuk tokenisasi pada tahap karakter.

Kita akan memilih pilihan kedua. `Tokenizer` juga boleh digunakan untuk tokenisasi kepada perkataan, jadi seseorang boleh beralih daripada tokenisasi tahap karakter kepada tahap perkataan dengan mudah.

Untuk melakukan tokenisasi pada tahap karakter, kita perlu memberikan parameter `char_level=True`:


In [2]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

tokenizer = keras.preprocessing.text.Tokenizer(char_level=True,lower=False)
tokenizer.fit_on_texts([x['title'].numpy().decode('utf-8') for x in ds_train])

Kami juga ingin menggunakan satu token khas untuk menandakan **akhir jujukan**, yang akan kami panggil `<eos>`. Mari tambahkannya secara manual ke dalam perbendaharaan kata:


In [3]:
eos_token = len(tokenizer.word_index)+1
tokenizer.word_index['<eos>'] = eos_token

vocab_size = eos_token + 1

Sekarang, untuk mengekod teks ke dalam urutan nombor, kita boleh menggunakan:


In [4]:
tokenizer.texts_to_sequences(['Hello, world!'])

[[48, 2, 10, 10, 5, 44, 1, 25, 5, 8, 10, 13, 78]]

## Melatih RNN Generatif untuk Menjana Tajuk

Cara kita akan melatih RNN untuk menjana tajuk berita adalah seperti berikut. Pada setiap langkah, kita akan mengambil satu tajuk, yang akan dimasukkan ke dalam RNN, dan untuk setiap aksara input, kita akan meminta rangkaian untuk menjana aksara output seterusnya:

![Imej menunjukkan contoh penjanaan RNN untuk perkataan 'HELLO'.](../../../../../translated_images/ms/rnn-generate.56c54afb52f9781d.webp)

Untuk aksara terakhir dalam urutan kita, kita akan meminta rangkaian untuk menjana token `<eos>`.

Perbezaan utama antara RNN generatif yang kita gunakan di sini adalah bahawa kita akan mengambil output dari setiap langkah RNN, dan bukan hanya dari sel terakhir. Ini boleh dicapai dengan menetapkan parameter `return_sequences` pada sel RNN.

Oleh itu, semasa latihan, input kepada rangkaian adalah urutan aksara yang telah dikodkan dengan panjang tertentu, dan output adalah urutan dengan panjang yang sama, tetapi digeser satu elemen dan diakhiri dengan `<eos>`. Minibatch akan terdiri daripada beberapa urutan seperti ini, dan kita perlu menggunakan **padding** untuk menyelaraskan semua urutan.

Mari kita cipta fungsi-fungsi yang akan mengubah dataset untuk kita. Oleh kerana kita ingin menambah padding pada tahap minibatch, kita akan terlebih dahulu mengelompokkan dataset dengan memanggil `.batch()`, dan kemudian `map` untuk melakukan transformasi. Jadi, fungsi transformasi akan mengambil keseluruhan minibatch sebagai parameter:


In [5]:
def title_batch(x):
    x = [t.numpy().decode('utf-8') for t in x]
    z = tokenizer.texts_to_sequences(x)
    z = tf.keras.preprocessing.sequence.pad_sequences(z)
    return tf.one_hot(z,vocab_size), tf.one_hot(tf.concat([z[:,1:],tf.constant(eos_token,shape=(len(z),1))],axis=1),vocab_size)

Beberapa perkara penting yang kita lakukan di sini:
* Kita mula-mula mengekstrak teks sebenar daripada tensor string
* `text_to_sequences` menukar senarai string kepada senarai tensor integer
* `pad_sequences` menambah padding pada tensor tersebut sehingga panjang maksimum
* Akhirnya, kita melakukan pengekodan satu-hot untuk semua aksara, serta melakukan pergeseran dan penambahan `<eos>`. Kita akan segera melihat mengapa kita memerlukan aksara yang dikodkan satu-hot

Namun, fungsi ini adalah **Pythonic**, iaitu ia tidak boleh diterjemahkan secara automatik ke dalam graf pengiraan Tensorflow. Kita akan mendapat ralat jika cuba menggunakan fungsi ini secara langsung dalam fungsi `Dataset.map`. Kita perlu membungkus panggilan Pythonic ini dengan menggunakan pembungkus `py_function`:


In [6]:
def title_batch_fn(x):
    x = x['title']
    a,b = tf.py_function(title_batch,inp=[x],Tout=(tf.float32,tf.float32))
    return a,b

> **Nota**: Membezakan antara fungsi transformasi Pythonic dan Tensorflow mungkin kelihatan agak kompleks, dan anda mungkin tertanya-tanya mengapa kita tidak mengubah dataset menggunakan fungsi Python standard sebelum menghantarnya ke `fit`. Walaupun ini sememangnya boleh dilakukan, menggunakan `Dataset.map` mempunyai kelebihan besar, kerana saluran transformasi data dilaksanakan menggunakan graf pengiraan Tensorflow, yang memanfaatkan pengiraan GPU, dan meminimumkan keperluan untuk memindahkan data antara CPU/GPU.

Sekarang kita boleh membina rangkaian penjana kita dan memulakan latihan. Ia boleh berdasarkan mana-mana sel berulang yang telah kita bincangkan dalam unit sebelumnya (simple, LSTM atau GRU). Dalam contoh kita, kita akan menggunakan LSTM.

Oleh kerana rangkaian mengambil aksara sebagai input, dan saiz kosa kata agak kecil, kita tidak memerlukan lapisan embedding, input yang dikodkan satu-panas (one-hot-encoded) boleh terus masuk ke dalam sel LSTM. Lapisan output akan menjadi pengklasifikasi `Dense` yang akan menukar output LSTM kepada nombor token yang dikodkan satu-panas.

Selain itu, memandangkan kita berurusan dengan jujukan panjang berubah, kita boleh menggunakan lapisan `Masking` untuk mencipta topeng yang akan mengabaikan bahagian rentetan yang dipadatkan. Ini tidak semestinya diperlukan, kerana kita tidak begitu berminat dengan segala-galanya yang melangkaui token `<eos>`, tetapi kita akan menggunakannya demi mendapatkan pengalaman dengan jenis lapisan ini. `input_shape` akan menjadi `(None, vocab_size)`, di mana `None` menunjukkan jujukan panjang berubah, dan bentuk output juga `(None, vocab_size)`, seperti yang anda boleh lihat daripada `summary`:


In [7]:
model = keras.models.Sequential([
    keras.layers.Masking(input_shape=(None,vocab_size)),
    keras.layers.LSTM(128,return_sequences=True),
    keras.layers.Dense(vocab_size,activation='softmax')
])

model.summary()
model.compile(loss='categorical_crossentropy')

model.fit(ds_train.batch(8).map(title_batch_fn))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, None, 84)          0         
_________________________________________________________________
lstm (LSTM)                  (None, None, 128)         109056    
_________________________________________________________________
dense (Dense)                (None, None, 84)          10836     
Total params: 119,892
Trainable params: 119,892
Non-trainable params: 0
_________________________________________________________________
15000/15000 [==============================] - 229s 15ms/step - loss: 1.5385


## Menjana output

Sekarang kita telah melatih model, kita ingin menggunakannya untuk menghasilkan beberapa output. Pertama sekali, kita memerlukan cara untuk menyahkod teks yang diwakili oleh urutan nombor token. Untuk melakukan ini, kita boleh menggunakan fungsi `tokenizer.sequences_to_texts`; namun, ia tidak berfungsi dengan baik dengan pengekodan token peringkat aksara. Oleh itu, kita akan mengambil kamus token daripada tokenizer (dipanggil `word_index`), membina peta terbalik, dan menulis fungsi penyahkodan kita sendiri:


In [10]:
reverse_map = {val:key for key, val in tokenizer.word_index.items()}

def decode(x):
    return ''.join([reverse_map[t] for t in x])

Sekarang, mari kita lakukan penjanaan. Kita akan bermula dengan beberapa string `start`, mengekodnya ke dalam urutan `inp`, dan kemudian pada setiap langkah kita akan memanggil rangkaian kita untuk menyimpulkan aksara seterusnya.

Output daripada rangkaian `out` adalah vektor dengan elemen sebanyak `vocab_size` yang mewakili kebarangkalian setiap token, dan kita boleh mencari nombor token yang paling mungkin dengan menggunakan `argmax`. Kemudian, kita tambahkan aksara ini ke senarai token yang dijana, dan teruskan proses penjanaan. Proses ini, iaitu penjanaan satu aksara, diulang sebanyak `size` kali untuk menjana jumlah aksara yang diperlukan, dan kita akan berhenti lebih awal apabila `eos_token` ditemui.


In [12]:
def generate(model,size=100,start='Today '):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            nc = tf.argmax(out)
            if nc==eos_token:
                break
            chars.append(nc.numpy())
            inp = inp+[nc]
        return decode(chars)
    
generate(model)

'Today #39;s lead to strike for the strike for the strike for the strike (AFP)'

## Menyampel output semasa latihan

Oleh kerana kita tidak mempunyai metrik berguna seperti *ketepatan*, satu-satunya cara untuk melihat bahawa model kita semakin baik adalah dengan **menyampel** string yang dihasilkan semasa latihan. Untuk melakukannya, kita akan menggunakan **callback**, iaitu fungsi yang boleh kita serahkan kepada fungsi `fit`, dan yang akan dipanggil secara berkala semasa latihan.


In [13]:
sampling_callback = keras.callbacks.LambdaCallback(
  on_epoch_end = lambda batch, logs: print(generate(model))
)

model.fit(ds_train.batch(8).map(title_batch_fn),callbacks=[sampling_callback],epochs=3)

Epoch 1/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.2703
Today #39;s a lead in the company for the strike
Epoch 2/3
15000/15000 [==============================] - 227s 15ms/step - loss: 1.2057
Today #39;s the Market Service on Security Start (AP)
Epoch 3/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.1752
Today #39;s a line on the strike to start for the start


Contoh ini sudah menghasilkan teks yang cukup baik, tetapi ia boleh diperbaiki lagi dalam beberapa cara:

* **Lebih banyak teks**. Kita hanya menggunakan tajuk untuk tugas kita, tetapi anda mungkin ingin mencuba dengan teks penuh. Ingat bahawa RNN tidak terlalu bagus dalam mengendalikan urutan panjang, jadi adalah masuk akal untuk membahagikannya kepada ayat yang lebih pendek, atau sentiasa melatih pada panjang urutan tetap dengan nilai yang telah ditetapkan `num_chars` (contohnya, 256). Anda boleh cuba mengubah contoh di atas kepada seni bina seperti ini, menggunakan [tutorial rasmi Keras](https://keras.io/examples/generative/lstm_character_level_text_generation/) sebagai inspirasi.

* **LSTM berlapis-lapis**. Adalah masuk akal untuk mencuba 2 atau 3 lapisan sel LSTM. Seperti yang disebutkan dalam unit sebelumnya, setiap lapisan LSTM mengekstrak corak tertentu daripada teks, dan dalam kes penjana peringkat aksara, kita boleh menjangkakan lapisan LSTM yang lebih rendah bertanggungjawab untuk mengekstrak suku kata, dan lapisan yang lebih tinggi - untuk perkataan dan kombinasi perkataan. Ini boleh dilaksanakan dengan mudah dengan memberikan parameter bilangan lapisan kepada pembina LSTM.

* Anda juga mungkin ingin bereksperimen dengan **unit GRU** dan melihat mana yang memberikan prestasi lebih baik, serta dengan **saiz lapisan tersembunyi yang berbeza**. Lapisan tersembunyi yang terlalu besar mungkin menyebabkan overfitting (contohnya, rangkaian akan belajar teks secara tepat), dan saiz yang lebih kecil mungkin tidak menghasilkan keputusan yang baik.


## Penjanaan teks lembut dan suhu

Dalam definisi `generate` sebelum ini, kita sentiasa memilih aksara dengan kebarangkalian tertinggi sebagai aksara seterusnya dalam teks yang dijana. Ini menyebabkan teks sering "berulang" antara urutan aksara yang sama berulang kali, seperti dalam contoh ini:
```
today of the second the company and a second the company ...
```

Namun, jika kita melihat pada taburan kebarangkalian untuk aksara seterusnya, mungkin perbezaan antara beberapa kebarangkalian tertinggi tidak begitu besar, contohnya satu aksara boleh mempunyai kebarangkalian 0.2, manakala yang lain - 0.19, dan sebagainya. Sebagai contoh, apabila mencari aksara seterusnya dalam urutan '*play*', aksara seterusnya boleh jadi sama ada ruang, atau **e** (seperti dalam perkataan *player*).

Ini membawa kita kepada kesimpulan bahawa tidak selalu "adil" untuk memilih aksara dengan kebarangkalian lebih tinggi, kerana memilih yang kedua tertinggi masih boleh menghasilkan teks yang bermakna. Lebih bijak untuk **sampel** aksara daripada taburan kebarangkalian yang diberikan oleh output rangkaian.

Pensampelan ini boleh dilakukan menggunakan fungsi `np.multinomial` yang melaksanakan apa yang dipanggil **taburan multinomial**. Fungsi yang melaksanakan penjanaan teks **lembut** ini ditakrifkan di bawah:


In [33]:
def generate_soft(model,size=100,start='Today ',temperature=1.0):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            probs = tf.exp(tf.math.log(out)/temperature).numpy().astype(np.float64)
            probs = probs/np.sum(probs)
            nc = np.argmax(np.random.multinomial(1,probs,1))
            if nc==eos_token:
                break
            chars.append(nc)
            inp = inp+[nc]
        return decode(chars)

words = ['Today ','On Sunday ','Moscow, ','President ','Little red riding hood ']
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"\n--- Temperature = {i}")
    for j in range(5):
        print(generate_soft(model,size=300,start=words[j],temperature=i))


--- Temperature = 0.3
Today #39;s strike #39; to start at the store return
On Sunday PO to Be Data Profit Up (Reuters)
Moscow, SP wins straight to the Microsoft #39;s control of the space start
President olding of the blast start for the strike to pay &lt;b&gt;...&lt;/b&gt;
Little red riding hood ficed to the spam countered in European &lt;b&gt;...&lt;/b&gt;

--- Temperature = 0.8
Today countie strikes ryder missile faces food market blut
On Sunday collores lose-toppy of sale of Bullment in &lt;b&gt;...&lt;/b&gt;
Moscow, IBM Diffeiting in Afghan Software Hotels (Reuters)
President Ol Luster for Profit Peaced Raised (AP)
Little red riding hood dace on depart talks #39; bank up

--- Temperature = 1.0
Today wits House buiting debate fixes #39; supervice stake again
On Sunday arling digital poaching In for level
Moscow, DS Up 7, Top Proble Protest Caprey Mamarian Strike
President teps help of roubler stepted lessabul-Dhalitics (AFP)
Little red riding hood signs on cash in Carter-youb

---

KeyError: 0

Kami telah memperkenalkan satu lagi parameter yang dipanggil **temperature**, yang digunakan untuk menunjukkan sejauh mana kita harus berpegang kepada kebarangkalian tertinggi. Jika temperature adalah 1.0, kita melakukan pensampelan multinomial yang adil, dan apabila temperature meningkat ke infiniti - semua kebarangkalian menjadi sama, dan kita memilih aksara seterusnya secara rawak. Dalam contoh di bawah, kita dapat melihat bahawa teks menjadi tidak bermakna apabila kita meningkatkan temperature terlalu tinggi, dan ia menyerupai teks "berkitar" yang dihasilkan secara keras apabila ia menjadi lebih dekat kepada 0.



---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan perkhidmatan terjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Walaupun kami berusaha untuk memastikan ketepatan, sila ambil maklum bahawa terjemahan automatik mungkin mengandungi kesilapan atau ketidaktepatan. Dokumen asal dalam bahasa asalnya harus dianggap sebagai sumber yang berwibawa. Untuk maklumat penting, terjemahan manusia profesional adalah disyorkan. Kami tidak bertanggungjawab atas sebarang salah faham atau salah tafsir yang timbul daripada penggunaan terjemahan ini.
